## Map/Reduce

* A structured programming model for data parallelism
  * Geared toward data intensive applications, not compute.
  * Data-parallel, distributed-memory, data-intensive  
* Map/Reduce is co-deployed with The **Google File System**
  * Large scale parallel file system for large objects
  * M/R requires such a global data service
* Open-source re-implementation
  * HADOOP: Map/Reduce programming
  * HDFS: Haddop Distributed File System
  
  
### Map/Reduce in Context

There is a nice [history](https://towardsdatascience.com/2003-2023-a-brief-history-of-big-data-25712351a6bc) of Google Map/Reduce, Hadoop!, and the big data ecosystem they created.

Map/Reduce from Google was the first data-driven, scalable, asynchronous data framework.  It was revolutionary:
  * ability to program 10,000+ computers
  * make program progress in the presence of arbitrary failures
  
This is in contrast to supercomputing at the time:
  * synchronous computing among all nodes
  * failures require all-program restart (typically)
  * compute-focused, rather than data oriented.
  

### Some important properties

* Automated parallelism
  * programmer has no interaction with number of processes workers.
  * same program runs on any M/R cluster
* Functional programming concepts
  * break progam into a data parallel portion (`map`) and a reduction (`reduce`)
  * user-defined (procedural) functions for each task executed in a functional framework
  * No *working memory* in either mappers or reducers. The functions are created and applied on each data element.
* Sequential/streaming data access
  * data are accessed from disk to mappers
  * mapper outputs a stream (of keys and values)
  * stream shuffled to reducers
  * reducers output to file system

  
<img src="https://cdn-images-1.medium.com/max/1600/1*KKm4roOpsum147kKk5qp7A.jpeg" width=768 />

### Key/Value

* All M/R data are key/value pairs
    * keys are interpreted by the M/R system. Must be sortable.
    * values are not interpreted by M/R system. Only used in functions. 
* Colors in this figure represent keys
  * that are not changed by mapper (in this example)
  * shuffled by the engine and delivered to reducer

### Map/Reduce Example

Wordcount example from the original Google paper. Produce a count of the occurrence of each word in a set of documents.

```python
    map ( String key, String value ):
        // key: document name (file name)
        // value: document contents
        for each word w in value:
            EmitIntermediate ( word, "1" );
```

Mapper outputs `key=word, value="1"` for each word. Note that the output key and input key are different.
  * map is a **transformation** of the key/value schema
  * the only effect of the function is the emtited key, value pair.

```python
    reduce ( String key, Iterator values ):
        // key: a word
        // value: a list of counts
        int result = 0;
        for each word v in values:
            result += ParsseInt ( v );
        EmitAsString ( result );
```
Reducer sums the counts of words.  Some properties:
  * reducer gets a list of values at a key
  * reduce cannot change the key, emits a value, that is reduced from list
  * user defined function
  
### WordCount Visualized

<img src="https://www.researchgate.net/profile/Oscar_Pereira3/publication/270448794/figure/fig6/AS:295098651824130@1447368409317/Word-count-program-flow-executed-with-MapReduce-5.png" width=768 title="from Oscar Perreira @ ResearchGate" />

### The Map/Reduce Sorting Guarantee

* Map: extracts a sorting key from the value
$$
 \langle key, value \rangle -> \langle sort\_key, output\_value \rangle
$$
* Shuffle does not sort strictly:
  * it route's to reducer based on sort key.
  * typically hashing maps key to reducer
  * Keys are sorted as they are presented to the reducer

Here is the guarantee:

> We guarantee that within a given partition, the intermediate key/value pairs are processed in increasing key order. This ordering guarantee makes it easy to generate
a sorted output file per partition, which is useful when
the output file format needs to support efficient random
access lookups by key, or users of the output find it convenient to have the data sorted._

The ordering guarantees sort within partitions. To sort completely send all output to a single partition (use one reducer).

### What can you do?

It seems like a limited programming framework.  But, many string processing programs fit naturally in this model.
  * Grep
  * Web-log processing
  * Reverse web-link graph
  * Term vectors per host
  * Inverted index
  * Distributed sort